# Remaining issues:
## Handled issues:
- [solved] distance between geometries is in degrees, we need it in meters to determine sailing durations
- [solved] need to check resource requests closely to check why sharing ships isn't working yet
- [solved] need to add resource requests to sites
- [solved] need to add real time (plus a specific starting date)
- [solved] need to add logging to Sites to lists volumes at different timesteps (this helps to animate on Google Earth)

## Open issues:
- [open] need to add sensitivity to weather (see work Joris den Uijl)
- [open] need to add soil characteristics and turbidity generation (see work Joris den Uijl)
- [open] need to add routing via routing graph (Dijkstra algorithm)
- [open] need to make case handling web based (quick setup & quick case comparison)
- [open] need to collect the code in a package
- [open] change processing so that it can handle rate_in and rate_out in stead of just rate


# Create necessary classes

In [1]:
# package(s) related to time, space and id
import datetime
import platform

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import geojson
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np

# digital twin package
from  digital_twin.core import Identifiable, Locatable, Log, HasContainer, HasResource, Movable, ContainerDependentMovable, Processor, SimpyObject

In [2]:
class Site(Identifiable, Locatable, Log, HasContainer, HasResource):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


class TransportResource(Identifiable, Log, ContainerDependentMovable, HasResource):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


class TransportProcessingResource(Identifiable, Log, ContainerDependentMovable, Processor, HasResource):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


class ProcessingResource(Identifiable, Locatable, Log, Processor, HasResource):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

## Activities

In [3]:
class Installation(Identifiable, SimpyObject):
    """The Installation Class forms a spefic class of activities with associated methods that can 
    initiate and suspend processes according to a number of specified conditions. This class deals 
    with transport and installation/placement of discrete and continuous objects.
    
    condition: expression that states when to initiate or to suspend activity
    origin: object with simpy Container from which to get (can be Site or Vessel)
    destination: object with simpy Container in which to put (can be Site or Vessel)
    loader: gets amount from 'origin' Container and puts it into 'mover' Container
    mover: moves amount in Container from 'origin' to 'destination'
    unloader: gets amount from 'mover' Container and puts it into 'destination' Container"""

    def __init__(self,
                 condition,
                 origin, destination,  
                 loader, mover, unloader,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        
        self.condition = condition
        self.origin = origin
        self.destination = destination
        self.loader = loader
        self.mover = mover
        self.unloader = unloader
        
        self.standing_by_proc = self.env.process(
            self.standing_by(condition, destination))
        self.installation_proc = self.env.process(
            self.installation_process_control(condition,
                             origin, destination,
                             loader, mover, unloader))
        self.installation_reactivate = self.env.event()

    def standing_by(self, condition, destination,):
        """Standing by"""
        shown = False

        while not eval(condition):
            if not shown:
                print('T=' + '{:06.2f}'.format(self.env.now) + ' ' + self.name + ' to ' + destination.name + ' suspended')
                shown = True
            yield self.env.timeout(3600) # step 3600 time units ahead

        print('T=' + '{:06.2f}'.format(self.env.now) + ' ' + 'Condition: ' + condition + ' is satisfied')

        self.installation_reactivate.succeed()  # "reactivate"
        self.installation_reactivate = self.env.event()
 
    def installation_process_control(self, condition,
                                           origin, destination,
                                           loader, mover, unloader):
        """Installation process control"""  
        while not eval(condition):
            yield self.installation_reactivate

        print('T=' + '{:06.2f}'.format(self.env.now) + ' '+ self.name + ' to ' + destination.name + ' started')
        while eval(condition):
            yield from self.installation_process(origin, destination,
                                                 loader, mover, unloader)

    def installation_process(self, origin, destination,
                                   loader, mover, unloader):
        """Installation process"""
        # estimate amount that should be transported
        amount = min(
            mover.container.capacity - mover.container.level,
            origin.container.level,
            origin.container.capacity - origin.total_requested,
            destination.container.capacity - destination.container.level,
            destination.container.capacity - destination.total_requested)
        
        if amount>0:
            # request access to the transport_resource
            origin.total_requested += amount
            destination.total_requested += amount
            if id(loader) == id(mover): 
                # this is the case when a hopper is used
                print('Using Hopper to process ' + str(amount))
                with mover.resource.request() as my_mover_turn:
                    yield my_mover_turn

                    # request access to the load_resource
                    mover.log_entry('loading start', self.env.now, mover.container.level)
                    yield from loader.process(origin, mover, amount, destination_resource_request=my_mover_turn)
                    mover.log_entry('loading stop', self.env.now, mover.container.level)

                    print('Loaded:')
                    print('  from:           ' + origin.name + ' contains: ' + str(origin.container.level))
                    print('  by:             ' + mover.name + ' contains: ' + str(mover.container.level))
                    print('  to:             ' + destination.name + ' contains: ' + str(destination.container.level))

                    old_location = mover.geometry

                    mover.log_entry('sailing full start', self.env.now, mover.container.level)
                    yield from mover.move(destination)
                    mover.log_entry('sailing full stop', self.env.now, mover.container.level)
                    
                    print('Moved:')
                    print('  from:            ' + format(old_location.x,'02.0f') + ' ' + format(old_location.x, '02.0f'))
                    print('  to:              ' + format(mover.geometry.x, '02.0f') + ' ' + format(mover.geometry.y, '02.0f'))

                    # request access to the placement_resource
                    mover.log_entry('unloading start', self.env.now, mover.container.level)
                    yield from unloader.process(mover, destination, amount, origin_resource_request=my_mover_turn)
                    mover.log_entry('unloading stop', self.env.now, mover.container.level)

                    print('Unloaded:')
                    print('  from:           ' + destination.name + ' contains: ' + str(destination.container.level))
                    print('  by:             ' + mover.name + ' contains: ' + str(mover.container.level))
                    print('  to:             ' + origin.name + ' contains: ' + str(origin.container.level))
                    
                    old_location = mover.geometry

                    mover.log_entry('sailing full start', self.env.now, mover.container.level)
                    yield from mover.move(origin)
                    mover.log_entry('sailing full stop', self.env.now, mover.container.level)
                    
                    print('Moved:')
                    print('  from:            ' + format(old_location.x,'02.0f') + ' ' + format(old_location.x, '02.0f'))
                    print('  to:              ' + format(mover.geometry.x, '02.0f') + ' ' + format(mover.geometry.y, '02.0f'))

                    # once a mover is assigned to an Activity it completes a full cycle
                    mover.resource.release(my_mover_turn)
            else: 
                # if not a hopper is used we have to handle resource requests differently
                print('Using Transport to process ' + str(amount))
                with mover.resource.request() as my_mover_turn:
                    yield my_mover_turn

                    # request access to the load_resource
                    with loader.resource.request() as my_load_resource_turn:
                        yield my_load_resource_turn

                        mover.log_entry('loading start', self.env.now, mover.container.level)
                        yield from loader.process(origin, mover, amount, destination_resource_request=my_mover_turn)
                        mover.log_entry('loading stop', self.env.now, mover.container.level)

                        print('Loaded:')
                        print('  from:           ' + origin.name + ' contains: ' + str(origin.container.level))
                        print('  by:             ' + mover.name + ' contains: ' + str(mover.container.level))
                        print('  to:             ' + destination.name + ' contains: ' + str(destination.container.level))
                    
                    old_location = mover.geometry

                    mover.log_entry('sailing full start', self.env.now, mover.container.level)
                    yield from mover.move(destination)
                    mover.log_entry('sailing full stop', self.env.now, mover.container.level)
                    
                    print('Moved:')
                    print('  from:            ' + format(old_location.x,'02.0f') + ' ' + format(old_location.x, '02.0f'))
                    print('  to:              ' + format(mover.geometry.x, '02.0f') + ' ' + format(mover.geometry.y, '02.0f'))

                    # request access to the placement_resource
                    with unloader.resource.request() as my_unloader_turn:
                        yield my_unloader_turn
                        
                        print('unloading')

                        mover.log_entry('unloading start', self.env.now, mover.container.level)
                        yield from unloader.process(mover, destination, amount, origin_resource_request=my_mover_turn)
                        mover.log_entry('unloading stop', self.env.now, mover.container.level)

                        print('Unloaded:')
                        print('  from:           ' + destination.name + ' contains: ' + str(destination.container.level))
                        print('  by:             ' + mover.name + ' contains: ' + str(mover.container.level))
                        print('  to:             ' + origin.name + ' contains: ' + str(origin.container.level))

                        unloader.resource.release(my_unloader_turn)

                    old_location = mover.geometry
                    
                    mover.log_entry('sailing full start', self.env.now, mover.container.level)
                    yield from mover.move(origin)
                    mover.log_entry('sailing full stop', self.env.now, mover.container.level)
                    
                    print('Moved:')
                    print('  from:            ' + format(old_location.x,'02.0f') + ' ' + format(old_location.x, '02.0f'))
                    print('  to:              ' + format(mover.geometry.x, '02.0f') + ' ' + format(mover.geometry.y, '02.0f'))

                    # once a mover is assigned to an Activity it completes a full cycle
                    mover.resource.release(my_mover_turn)
        else:
            print('Nothing to move')
            yield self.env.timeout(3600)
            

# Start case

In [4]:
# *** Create a project environment
my_env = simpy.Environment()
start = my_env.now

In [5]:
# simulation returns time in seconds with epoch as the reference
my_env.epoch = datetime.datetime.now()

## Define sites

In [6]:
Sites = []
# *** Generate stock sites
# - sites in database
data_stock_01 = {"env": my_env,
                "name": "Stock 01", "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),
                "capacity": 150000, "level": 150000}
data_stock_02 = {"env": my_env,
                "name": "Stock 02", "geometry": shapely.geometry.Point(5.271417603333632, 52.9638452897506),
                "capacity": 150000, "level": 150000}
data_stock_03 = {"env": my_env,
                "name": "Stock 03", "geometry": shapely.geometry.Point(5.919298185633251, 52.94239823421129),
                "capacity": 150000, "level": 150000}
data_stock_04 = {"env": my_env,
                "name": "Stock 04", "geometry": shapely.geometry.Point(5.919298185633251, 52.94239823421129),
                "capacity": 150000, "level": 150000}

# - create site objects
stock_01 = Site(**data_stock_01)
Sites.append(stock_01)
stock_02 = Site(**data_stock_02)
Sites.append(stock_02)
stock_03 = Site(**data_stock_03)
Sites.append(stock_03)
stock_04 = Site(**data_stock_04)
Sites.append(stock_04)

# *** Generate placement sites
# - Clay layer
layer_name = '_clay'
capacity = 5000
level = 0
nums = 20
start = [5.054676856441372,52.94042293840172] # Den Oever 
stop = [5.294877712236641,53.06686424241725] # Kornwerderzand

# - generate a 'nums' amount of sites between the selected start and stop points
lats = np.linspace(start[0], stop[0], num=nums)
lons = np.linspace(start[1], stop[1],  num=nums)

# - option to create a range of sites between two points
for i in range(nums):
    # - sites in database (nr indicates km's from Den Oever haven)
    data_site = {"env": my_env,
                "name": "KP" + format(i,'02.0f') + layer_name, "geometry": shapely.geometry.Point(lats[i], lons[i]),
                "capacity": capacity, "level": level}
    
    # - create site objects
    vars()['Site_' + "KP" + format(i,'02.0f') + layer_name] = Site(**data_site)
    Sites.append(vars()['Site_' + "KP" + format(i,'02.0f') + layer_name])

# - Sand layer
layer_name = '_sand'
capacity = 5000
level = 0
nums = 20
start = [5.052051052879287,52.9421894472733] # Den Oever 
stop = [5.292216781509101,53.06886359869087] # Kornwerderzand

# - generate a 'nums' amount of sites between the selected start and stop points
lats = np.linspace(start[0], stop[0], num=nums)
lons = np.linspace(start[1], stop[1],  num=nums)

# - option to create a range of sites between two points
for i in range(nums):
    # - sites in database (nr indicates km's from Den Oever haven)
    data_site = {"env": my_env,
                "name": "KP" + format(i,'02.0f') + layer_name, "geometry": shapely.geometry.Point(lats[i], lons[i]),
                "capacity": capacity, "level": level}
    
    # - create site objects
    vars()['Site_' + "KP" + format(i,'02.0f') + layer_name] = Site(**data_site)
    Sites.append(vars()['Site_' + "KP" + format(i,'02.0f') + layer_name])

# - Armour layer
layer_name = '_armour'
capacity = 5000
level = 0
nums = 20
start = [5.049510554598302,52.94393628899332] # Den Oever 
stop = [5.289636346490858,53.07053144816584] # Kornwerderzand

# - generate a 'nums' amount of sites between the selected start and stop points
lats = np.linspace(start[0], stop[0], num=nums)
lons = np.linspace(start[1], stop[1],  num=nums)

# - option to create a range of sites between two points
for i in range(nums):
    # - sites in database (nr indicates km's from Den Oever haven)
    data_site = {"env": my_env,
                "name": "KP" + format(i,'02.0f') + layer_name, "geometry": shapely.geometry.Point(lats[i], lons[i]),
                "capacity": capacity, "level": level}
    
    # - create site objects
    vars()['Site_' + "KP" + format(i,'02.0f') + layer_name] = Site(**data_site)
    Sites.append(vars()['Site_' + "KP" + format(i,'02.0f') + layer_name])

# - Levvel layer
layer_name = '_levvel'
capacity = 5000
level = 0
nums = 20
start = [5.046556507026805,52.94579445406793] # Den Oever 
stop = [5.286775240694118,53.07264015015531] # Kornwerderzand

# - generate a 'nums' amount of sites between the selected start and stop points
lats = np.linspace(start[0], stop[0], num=nums)
lons = np.linspace(start[1], stop[1],  num=nums)

# - option to create a range of sites between two points
for i in range(nums):
    # - sites in database (nr indicates km's from Den Oever haven)
    data_site = {"env": my_env,
                "name": "KP" + format(i,'02.0f') + layer_name, "geometry": shapely.geometry.Point(lats[i], lons[i]),
                "capacity": capacity, "level": level}
    
    # - create site objects
    vars()['Site_' + "KP" + format(i,'02.0f') + layer_name] = Site(**data_site)
    Sites.append(vars()['Site_' + "KP" + format(i,'02.0f') + layer_name])

## Define equipment

In [7]:
# *** Define fleet

# sites in database (nr indicates km's from Den Oever haven)
# - processing resources
def compute_v_provider(v_empty, v_full):
    return lambda x: x * (v_full - v_empty) + v_empty


data_gantry_crane = {"env": my_env,
                "name": "Gantry crane", "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),
                "rate": 0.10, "nr_resources": 1}
data_installation_crane = {"env": my_env,
                "name": "Installation crane", "geometry": shapely.geometry.Point(5.197016484858931, 53.0229621352376),
                "rate": 0.05, "nr_resources": 1}

# - transport resources
data_transport_barge_01 = {"env": my_env,
                "name": "Transport barge 01", "geometry": shapely.geometry.Point(5.070195628786471, 52.93917167503315),
                "capacity": 1000, "level": 0, "nr_resources": 1, "compute_v": compute_v_provider(1.6, 1)}
data_transport_barge_02 = {"env": my_env,
                "name": "Transport barge 02", "geometry": shapely.geometry.Point(5.070195628786471, 52.93917167503315),
                "capacity": 1000, "level": 0, "nr_resources": 1, "compute_v": compute_v_provider(1.6, 1)}

# - transport processing resources
data_hopper = {"env": my_env,
                "name": "Hopper", "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),
                "rate": 2, "nr_resources": 1, "capacity": 1000, "level": 0, "compute_v": compute_v_provider(2, 1.5)}

# create site objects
# - processing resources
gantry_crane = ProcessingResource(**data_gantry_crane)
installation_crane = ProcessingResource(**data_installation_crane)

# - transport resources
transport_barge_01 = TransportResource(**data_transport_barge_01)
transport_barge_02 = TransportResource(**data_transport_barge_02)

# - transport processing resources
hopper = TransportProcessingResource(**data_hopper)

## Define activities

In [8]:
# *** Define installation activities
transport_barges=[]
transport_barges.append(transport_barge_01)
transport_barges.append(transport_barge_02)

# Clay
layer_name = '_clay'
for i in range(nums):
    for transport_barge in transport_barges:
        # - sites in database (nr i indicates km's from Den Oever haven)
        if i==0:
            condition = "'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000'''"
            data_act = {"env": my_env,
                    "name": "Clay placement",
                    "origin": stock_01, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}
        else:
            condition = "'''" + eval("'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000'''") + \
                        ' and ' + eval("'''Site_KP" + format(i-1,'02.0f') + layer_name + ".container.level==5000'''") + "'''"
            data_act = {"env": my_env,
                    "name": "Clay placement",
                    "origin": stock_01, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}

        # - create site objects
        vars()['Act_' + format(i,'02.0f') + layer_name] = Installation(**data_act)

# Sand
layer_name = '_sand'
previous_layer_name = '_clay'
for i in range(nums):
    # - sites in database (nr i indicates km's from Den Oever haven)
    if i==0:
        condition =  "'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000" + \
                    ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
        data_act = {"env": my_env,
                "name": "Sand placement",
                "origin": stock_02, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                "loader": hopper, "mover": hopper, "unloader": hopper,
                "condition": eval(condition)}
    else:
        condition = "'''" + eval("'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000'''") + \
                    ' and ' + eval("'''Site_KP" + format(i-1,'02.0f') + layer_name + ".container.level==5000'''") + \
                    ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
        data_act = {"env": my_env,
                "name": "Sand placement",
                "origin": stock_02, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                "loader": hopper, "mover": hopper, "unloader": hopper,
                "condition": eval(condition)}

    # - create site objects
    vars()['Act_' + format(i,'02.0f') + layer_name] = Installation(**data_act)

# Armour
layer_name = '_armour'
previous_layer_name = '_sand'
for i in range(nums):
    for transport_barge in transport_barges:
        # - sites in database (nr i indicates km's from Den Oever haven)
        if i==0:
            condition =  "'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000" + \
                        ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
            data_act = {"env": my_env,
                    "name": "Armour placement",
                    "origin": stock_03, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}
        else:
            condition = "'''" + eval("'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000'''") + \
                    ' and ' + eval("'''Site_KP" + format(i-1,'02.0f') + layer_name + ".container.level==5000'''") + \
                    ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
            data_act = {"env": my_env,
                    "name": "Armour placement",
                    "origin": stock_03, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}

        # - create site objects
        vars()['Act_' + format(i,'02.0f') + layer_name] = Installation(**data_act)

# Levvel
layer_name = '_levvel'
previous_layer_name = '_armour'
for i in range(nums):
    for transport_barge in transport_barges:
    # - sites in database (nr i indicates km's from Den Oever haven)
        if i==0:
            condition =  "'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000" + \
                        ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
            data_act = {"env": my_env,
                    "name": "Block placement",
                    "origin": stock_04, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}
        else:
            condition = "'''" + eval("'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000'''") + \
                    ' and ' + eval("'''Site_KP" + format(i-1,'02.0f') + layer_name + ".container.level==5000'''") + \
                    ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
            data_act = {"env": my_env,
                    "name": "Block placement",
                    "origin": stock_04, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}
    
        # - create site objects
        vars()['Act_' + format(i,'02.0f') + layer_name] = Installation(**data_act)
            

## Run simulation

In [9]:
#*** Run the project
my_env.run()

T=000.00 Condition: Site_KP00_clay.container.level<5000 is satisfied
T=000.00 Clay placement to KP00_clay started
Using Transport to process 1000
T=000.00 Condition: Site_KP00_clay.container.level<5000 is satisfied
T=000.00 Clay placement to KP00_clay started
Using Transport to process 1000
T=000.00 Clay placement to KP01_clay suspended
T=000.00 Clay placement to KP01_clay suspended
T=000.00 Clay placement to KP02_clay suspended
T=000.00 Clay placement to KP02_clay suspended
T=000.00 Clay placement to KP03_clay suspended
T=000.00 Clay placement to KP03_clay suspended
T=000.00 Clay placement to KP04_clay suspended
T=000.00 Clay placement to KP04_clay suspended
T=000.00 Clay placement to KP05_clay suspended
T=000.00 Clay placement to KP05_clay suspended
T=000.00 Clay placement to KP06_clay suspended
T=000.00 Clay placement to KP06_clay suspended
T=000.00 Clay placement to KP07_clay suspended
T=000.00 Clay placement to KP07_clay suspended
T=000.00 Clay placement to KP08_clay suspended
T=0

  from:            05 05
  to:              05 53
Unloaded:
  from:           KP01_clay contains: 2000
  by:             Transport barge 01 contains: 0
  to:             Stock 01 contains: 142000
unloading
Unloaded:
  from:           KP00_sand contains: 4000
  by:             Hopper contains: 0
  to:             Stock 02 contains: 146000
Moved:
  from:            05 05
  to:              05 53
Using Transport to process 1000
Moved:
  from:            05 05
  to:              05 53
Using Hopper to process 1000
Loaded:
  from:           Stock 02 contains: 145000
  by:             Hopper contains: 1000
  to:             KP00_sand contains: 4000
Loaded:
  from:           Stock 01 contains: 141000
  by:             Transport barge 01 contains: 1000
  to:             KP01_clay contains: 3000
Moved:
  from:            05 05
  to:              05 53
Moved:
  from:            05 05
  to:              05 53
Unloaded:
  from:           KP00_sand contains: 5000
  by:             Hopper contains: 0

Moved:
  from:            06 06
  to:              05 53
unloading
Unloaded:
  from:           KP02_clay contains: 3000
  by:             Transport barge 01 contains: 0
  to:             Stock 01 contains: 137000
Moved:
  from:            06 06
  to:              05 53
unloading
Moved:
  from:            05 05
  to:              05 53
Using Transport to process 1000
Loaded:
  from:           Stock 03 contains: 143000
  by:             Transport barge 01 contains: 1000
  to:             KP01_armour contains: 1000
Moved:
  from:            05 05
  to:              05 53
Unloaded:
  from:           KP00_levvel contains: 1000
  by:             Transport barge 02 contains: 0
  to:             Stock 04 contains: 149000
unloading
Unloaded:
  from:           KP01_armour contains: 2000
  by:             Transport barge 01 contains: 0
  to:             Stock 03 contains: 143000
Moved:
  from:            05 05
  to:              06 53
Using Transport to process 1000
Loaded:
  from:           Stoc

Nothing to move
Moved:
  from:            06 06
  to:              05 53
unloading
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Unloaded:
  from:           KP03_clay contains: 1000
  by:             Transport barge 02 contains: 0
  to:             Stock 01 contains: 133000
Nothing to move
Moved:
  from:            06 06
  to:              05 53
unloading
Moved:
  from:            05 05
  to:              05 53
Using Transport to process 1000
Nothing to move
Nothing to move
Nothing to move
Loaded:
  from:           Stock 04 contains: 145000
  by:             Transport barge 02 contains: 1000
  to:             KP00_levvel contains: 4000
Nothing to move
Moved:
  from:            05 05
  to:              05 53
Nothing to move
Unloaded:
  from:           KP03_clay contains: 2000
  by:             Transport barge 01 contains: 0
  to:             Stock 01 contains: 133000
unloading
T=1220400.00 Condition: Site_KP01_levvel.container.level<5000 and Site_KP00_l

T=1897200.00 Condition: Site_KP02_levvel.container.level<5000 and Site_KP01_levvel.container.level==5000 and Site_KP02_armour.container.level==5000 is satisfied
T=1897200.00 Block placement to KP02_levvel started
Using Transport to process 1000
T=1897200.00 Block placement to KP02_levvel started
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Moved:
  from:            06 06
  to:              05 53
Unloaded:
  from:           KP01_levvel contains: 5000
  by:             Transport barge 01 contains: 0
  to:             Stock 04 contains: 140000
unloading
Using Transport to process 1000
Unloaded:
  from:           KP04_clay contains: 1000
  by:             Transport barge 02 contains: 0
  to:             Stock 01 contains: 129000
Moved:
  from:            05 05
  to:              05 53
Using Transport to process 1000
Loaded:
  from:           Stock 03 contains: 134000
  by:             Transport barge 02 contains: 1000
  to:             KP03_armour contain

...

  from:            05 05
  to:              05 53
Using Hopper to process 1000
Loaded:
  from:           Stock 02 contains: 73000
  by:             Hopper contains: 1000
  to:             KP15_sand contains: 1000
Moved:
  from:            05 05
  to:              05 53
Unloaded:
  from:           KP15_clay contains: 5000
  by:             Transport barge 01 contains: 0
  to:             Stock 01 contains: 70000
Unloaded:
  from:           KP15_sand contains: 2000
  by:             Hopper contains: 0
  to:             Stock 02 contains: 73000
Moved:
  from:            05 05
  to:              05 53
Using Hopper to process 1000
Loaded:
  from:           Stock 02 contains: 72000
  by:             Hopper contains: 1000
  to:             KP15_sand contains: 2000
Moved:
  from:            05 05
  to:              06 53
Nothing to move
Moved:
  from:            05 05
  to:              05 53
Moved:
  from:            05 05
  to:              05 53
Unloaded:
  from:           KP15_sand contain

  from:            05 05
  to:              05 53
unloading
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Moved:
  from:            06 06
  to:              05 53
Nothing to move
Unloaded:
  from:           KP16_armour contains: 1000
  by:             Transport barge 02 contains: 0
  to:             Stock 03 contains: 68000
unloading
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Unloaded:
  from:           KP16_armour contains: 2000
  by:             Transport barge 01 contains: 0
  to:             Stock 03 contains: 68000
Nothing to move
Nothing to move
Moved:
  from:            05 05
  to:              06 53
Using Transport to process 1000
Nothing to move
Nothing to move
Nothing to move
Loaded:
  from:           Stock 04 contains: 75000
  by:             Transport barge 02 contains: 1000
  to:             KP14_levvel contains: 4000
Nothing to move
Nothing to move
Nothing to move
Moved:
  from:            05 05
  to: 

Unloaded:
  from:           KP15_levvel contains: 4000
  by:             Transport barge 01 contains: 0
  to:             Stock 04 contains: 71000
Moved:
  from:            05 05
  to:              06 53
Using Transport to process 1000
Loaded:
  from:           Stock 03 contains: 64000
  by:             Transport barge 02 contains: 1000
  to:             KP17_armour contains: 0
Moved:
  from:            05 05
  to:              06 53
Nothing to move
Nothing to move
Nothing to move
Loaded:
  from:           Stock 01 contains: 58000
  by:             Transport barge 01 contains: 1000
  to:             KP18_clay contains: 1000
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Moved:
  from:            06 06
  to:              05 53
unloading
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Moved:
  from:            06 06
  to:              05 53
Unloaded:
  from:           KP17_arm

  to:             Stock 04 contains: 67000
Moved:
  from:            05 05
  to:              06 53
Using Transport to process 1000
Loaded:
  from:           Stock 04 contains: 66000
  by:             Transport barge 02 contains: 1000
  to:             KP16_levvel contains: 3000
Moved:
  from:            05 05
  to:              06 53
Using Transport to process 1000
Loaded:
  from:           Stock 03 contains: 58000
  by:             Transport barge 01 contains: 1000
  to:             KP18_armour contains: 1000
Moved:
  from:            06 06
  to:              05 53
unloading
Moved:
  from:            06 06
  to:              05 53
Unloaded:
  from:           KP16_levvel contains: 4000
  by:             Transport barge 02 contains: 0
  to:             Stock 04 contains: 66000
unloading
Unloaded:
  from:           KP18_armour contains: 2000
  by:             Transport barge 01 contains: 0
  to:             Stock 03 contains: 58000
Moved:
  from:            05 05
  to:              06 5

Loaded:
  from:           Stock 04 contains: 51000
  by:             Transport barge 01 contains: 1000
  to:             KP19_levvel contains: 3000
Unloaded:
  from:           KP19_levvel contains: 3000
  by:             Transport barge 02 contains: 0
  to:             Stock 04 contains: 51000
Moved:
  from:            05 05
  to:              06 53
Using Transport to process 1000
Moved:
  from:            06 06
  to:              05 53
unloading
Loaded:
  from:           Stock 04 contains: 50000
  by:             Transport barge 02 contains: 1000
  to:             KP19_levvel contains: 4000
Unloaded:
  from:           KP19_levvel contains: 4000
  by:             Transport barge 01 contains: 0
  to:             Stock 04 contains: 50000
Moved:
  from:            05 05
  to:              06 53
Nothing to move
Nothing to move
Moved:
  from:            06 06
  to:              05 53
unloading
Unloaded:
  from:           KP19_levvel contains: 5000
  by:             Transport barge 02 contai

## Some basic visualisation on Google Earth

In [10]:
icon = 'http://maps.google.com/mapfiles/kml/shapes/donut.png'
size = 1

kml = Kml()
fol = kml.newfolder(name="A Folder")

shared_style = Style()
shared_style.labelstyle.color = 'ffffffff'  # White
shared_style.labelstyle.scale = 1  
shared_style.iconstyle.color = 'ffff0000'  # Blue
shared_style.iconstyle.scale = 1
shared_style.iconstyle.icon.href = icon

for site in Sites:
    if not site.value or len(site.value) < 2:
        pnt = fol.newpoint(name=site.name, coords=[(site.geometry.x, site.geometry.y)])
        pnt.timestamp.when = my_env.epoch.isoformat()
        pnt.style = shared_style
    else:
        # ignore last point because we need an endpoint
        for i, value in enumerate(site.value[:-1]):
            # convert to real dates
            begin = my_env.epoch + datetime.timedelta(seconds=site.t[i])
            end = my_env.epoch + datetime.timedelta(seconds=site.t[i+1])
            pnt = fol.newpoint(name='', coords=[(site.geometry.x, site.geometry.y)])   
            # convert to string
            pnt.timespan.begin = begin.isoformat()
            pnt.timespan.end = end.isoformat()
            # use custom style if we are time dependent
            style = Style()
            style.labelstyle.color = 'ffffffff'  # White
            style.labelstyle.scale = 1  
            style.iconstyle.color = 'ffff0000'  # Blue
            style.iconstyle.scale = (value / site.container.capacity) * size
            style.iconstyle.icon.href = icon
            pnt.style = style
        begin = my_env.epoch + datetime.timedelta(seconds=site.t[-1])
        end = my_env.epoch + datetime.timedelta(seconds=my_env.now)
        pnt = fol.newpoint(name='', coords=[(site.geometry.x, site.geometry.y)])           
        # convert to string
        pnt.timespan.begin = begin.isoformat()
        pnt.timespan.end = end.isoformat()
        # use custom style if we are time dependent
        style = Style()
        style.labelstyle.color = 'ffffffff'  # White
        style.labelstyle.scale = 1  
        style.iconstyle.color = 'ff00ff00'   # Green
        style.iconstyle.scale = (site.value[-1] / site.container.capacity) * size
        style.iconstyle.icon.href = icon
        pnt.style = style

kml.save("sharedstyle.kml")

In [11]:
# open the file
if platform.system():
    !start ./sharedstyle.kml
else:
    !start explorer ./sharedstyle.kml

In [23]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
class plot:
    """plot class"""

    def vessel_planning(vessels, activities, colors, web=False):
        """create a plot of the planning of vessels"""

        def get_segments(series, activity, y_val):
            """extract 'start' and 'stop' of activities from log"""
            x = []
            y = []
            for i, v in series.iteritems():
                if v == 'start ' + activity:
                    start = i
                if v == 'stop ' + activity:
                    x.extend((start, start, i, i, i))
                    y.extend((y_val, y_val, y_val, y_val, None))
            return x, y

        # organise logdata into 'dataframes' 
        dataframes = []
        for vessel in vessels:
            df = pd.DataFrame(
                {'log_value': vessel.value, 'log_string': vessel.log}, vessel.t)
            dataframes.append(df)
        df = dataframes[0]

        # prepare traces for each of the activities
        traces = []
        for i, activity in enumerate(activities):
            x_combined = []
            y_combined = []
            for k, df in enumerate(dataframes):
                y_val = vessels[k].name
                x, y = get_segments(
                    df['log_string'], activity=activity, y_val=y_val)
                x_combined.extend(x)
                y_combined.extend(y)
            traces.append(go.Scatter(
                name=activity,
                x=x_combined,
                y=y_combined,
                mode='lines',
                hoverinfo='y+name',
                line=dict(color=colors[i], width=10),
                connectgaps=False))
        
        # prepare layout of figure
        layout = go.Layout(
            title='Vessel planning',
            hovermode='closest',
            legend=dict(x=0, y=-.2, orientation="h"),
            xaxis=dict(
                title='Time',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'),
                range=[0, vessel.t[-1]]),
            yaxis=dict(
                title='Vessels',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f')))
        
        # plot figure
        init_notebook_mode(connected=True)        
        fig = go.Figure(data=traces, layout=layout)
        return iplot(fig, filename='news-source')
    

In [24]:
vessels=[]
vessels.append(transport_barge_01)
vessels.append(transport_barge_02)

In [25]:
activities = ['loading', 'unloading', 'sailing full', 'sailing empty']
colors = {0:'rgb(55,126,184)', 1:'rgb(77,175,74)', 2:'rgb(255,0,0)', 3:'rgb(255,150,0)'}

plot.vessel_planning(vessels, activities, colors)